In [1]:
import os
import json
import re

In [2]:
def merge_details(backward_details):
    merged_list = []
    current_item = ""

    for i, item in enumerate(backward_details):
        if item.strip() and item.strip()[0].isdigit():
            j = 1
            while j < len(item.strip()) and item.strip()[j].isdigit():
                j += 1
            if j < len(item.strip()) and item.strip()[j] == "：":
                merged_list.append(current_item)
                current_item = item.strip()
            else:
                current_item += item.strip()
        else:
            current_item += item.strip()

    merged_list.append(current_item)
    merged_list.pop(0)
    return merged_list


def extract_tuples(string):
    tuples = []
    depth = 0
    start = None
    for i, char in enumerate(string):
        if char == "[" or char == '(' or char == '（':
            if depth == 0:
                start = i
            depth += 1
        elif char == "]" or char == ')' or char == '）':
            depth -= 1
            if depth == 0 and start is not None:
                tuples.append(string[start+1:i])
                start = None
    return tuples


def get_results(merged_list, input_list, entity_type_list, eve_extend_map, mode):
    input_list2ent = {}
    for idx, item in enumerate(input_list):
        input_list2ent[item] = entity_type_list[idx//10]

    eve_extend_index = 0
    cur_eve_extend_list = []
    A = []

    for idx, item in enumerate(merged_list):
        B = []
        cur_output_3 = item.split('Output:')
        cur_idx = cur_output_3[0].split('：')[0]
        cur_example = input_list[int(cur_idx)-1]
       
        cur_true_eve = input_list2ent[cur_example]

        if eve_extend_index == len(cur_eve_extend_list):
            eve_extend_index = 0

        cur_eve_extend_list = eve_extend_map[cur_true_eve]
        cur_extend_eve = cur_eve_extend_list[eve_extend_index]


        for output in cur_output_3[1:]:
            sep = ""
            if 'zh' in mode:
                output = output.replace(":", "：")
                if "生成事件检测列表:" in output:
                    sep = "生成事件检测列表:"
                elif "生成事件检测列表：" in output:
                    sep = "生成事件检测列表："
            elif 'en' in mode:
                if "Generate a list of event detections:" in output:
                    sep = "Generate a list of event detections:"
                elif "Generated event detection list:" in output:
                    sep = "Generated event detection list:"
                elif "Event detection list:" in output:
                    sep = "Event detection list:"
                elif "Event detections:" in output:
                    sep = "Event detections:"
                elif "event detection list:" in output:
                    sep = "event detection list:"
                elif "List of Event Detections:" in output:
                    sep = "List of Event Detections:"
                elif "generate a list of event detections:" in output:
                    sep = "generate a list of event detections:"
                elif "generate event detection list:" in output:
                    sep = "generate event detection list:"
                elif "event detection list:" in output:
                    sep = "event detection list:"
                # elif "Trigger span:" in output:
                #     sep = "Trigger span:"
                # elif "Trigger word span:" in output:
                #     sep = "Trigger word span:"
                # elif "Trigger Word:" in output:
                #     sep = "Trigger Word:"
                # elif "The trigger word for this case is " in output:
                #     sep = "The trigger word for this case is "
                # elif "The trigger word for this event type is " in output:
                #     sep = "The trigger word for this event type is "
            if sep:
                tmp = output.split(sep)[1]
                matches = extract_tuples(tmp)
                if matches:
                    for match in matches:
                        if 'zh' in mode:
                            triple = tuple(match.strip().replace('，',',').strip('（）').split(','))
                        elif 'en' in mode:
                            triple = tuple(match.strip().strip('()').split(','))
                        if len(triple) == 2:
                            B.append((int(cur_idx)-1, cur_extend_eve, triple[1].strip()))
                        else:
                            B.append((int(cur_idx)-1, cur_extend_eve, ''))
                else:
                    B.append((int(cur_idx)-1, cur_extend_eve, ''))
                
            else:
                B.append((int(cur_idx)-1, cur_extend_eve, ''))
                continue
        if B:
            A.append(B)    

        eve_extend_index += 1

    A = [list(set(item)) for item in A]
    
    with open(f'./results/backward_{mode}.txt','w',encoding='utf-8') as f:
        for item in A:
            f.write(str(item)+'\n')


def get_gold_results(merged_list, input_list, entity_type_list, mode):
    input_list2ent = {}
    for idx, item in enumerate(input_list):
        input_list2ent[item] = entity_type_list[idx//10]

    A = []

    for idx, item in enumerate(merged_list):
        B = []
        cur_output_3 = item.split('Output:')
        cur_idx = cur_output_3[0].split('：')[0]
        cur_example = input_list[int(cur_idx)-1]
        cur_true_eve = input_list2ent[cur_example]


        for output in cur_output_3[1:]:

            sep = ""
            if 'zh' in mode:
                output = output.replace(":", "：")
                if "生成事件检测列表:" in output:
                    sep = "生成事件检测列表:"
                elif "生成事件检测列表：" in output:
                    sep = "生成事件检测列表："
            elif 'en' in mode:
                if "Generate a list of event detections:" in output:
                    sep = "Generate a list of event detections:"
                elif "Generated event detection list:" in output:
                    sep = "Generated event detection list:"
                elif "Event detection list:" in output:
                    sep = "Event detection list:"
                elif "Event detections:" in output:
                    sep = "Event detections:"
                elif "event detection list:" in output:
                    sep = "event detection list:"
                elif "List of Event Detections:" in output:
                    sep = "List of Event Detections:"
                elif "generate a list of event detections:" in output:
                    sep = "generate a list of event detections:"
                elif "generate event detection list:" in output:
                    sep = "generate event detection list:"
                elif "event detection list:" in output:
                    sep = "event detection list:"
                # elif "Trigger span:" in output:
                #     sep = "Trigger span:"
                # elif "Trigger word span:" in output:
                #     sep = "Trigger word span:"
                # elif "Trigger Word:" in output:
                #     sep = "Trigger Word:"
                # elif "The trigger word for this case is " in output:
                #     sep = "The trigger word for this case is "
                # elif "The trigger word for this event type is " in output:
                #     sep = "The trigger word for this event type is "
            if sep:
                tmp = output.split(sep)[1]
                matches = extract_tuples(tmp)
                if matches:
                    for match in matches:
                        if 'zh' in mode:
                            triple = tuple(match.strip().replace('，',',').strip('（）').split(','))
                        elif 'en' in mode:
                            triple = tuple(match.strip().strip('()').split(','))
                        if len(triple) == 2:
                            B.append((int(cur_idx)-1, cur_true_eve, triple[1].strip()))
                        else:
                            B.append((int(cur_idx)-1, cur_true_eve, ''))
                else:
                    B.append((int(cur_idx)-1, cur_true_eve, ''))
                
            else:
                B.append((int(cur_idx)-1, cur_true_eve, ''))
                continue
        if B:
            A.append(B)    

    A = [list(set(item)) for item in A]
    
    with open(f'./results/backward_gold_{mode}.txt','w',encoding='utf-8') as f:
        for item in A:
            f.write(str(item)+'\n')

DuEE

In [5]:
with open('./results/details_zh.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_zh.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/duee/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/duee/final_event_extend_map_zh.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('../DuEE/processed_data/labels.txt','r',encoding='utf-8') as f:
    event_type_list = [item.strip() for item in f.readlines()]

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, event_type_list, eve_extend_map, 'zh')

merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, event_type_list, 'zh')

CASIE

In [4]:
with open('./results/details_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_en.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/casie/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/casie/final_event_extend_map_en.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('../CASIE/processed_data/labels.txt','r',encoding='utf-8') as f:
    event_type_list = [item.strip() for item in f.readlines()]

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, event_type_list, eve_extend_map, 'en')

merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, event_type_list, 'en')

Alpaca-DuEE

In [9]:
with open('./results/details_alpaca_33B_zh.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_alpaca_33B_zh.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/duee/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/duee/final_event_extend_map_alpaca_33B_zh.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('../DuEE/processed_data/labels.txt','r',encoding='utf-8') as f:
    event_type_list = [item.strip() for item in f.readlines()]

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, event_type_list, eve_extend_map, 'alpaca_33B_zh')

merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, event_type_list, 'alpaca_33B_zh')

Alpaca-CASIE

In [10]:
with open('./results/details_alpaca_33B_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_alpaca_33B_en.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/casie/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/casie/final_event_extend_map_alpaca_33B_en.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('../CASIE/processed_data/labels.txt','r',encoding='utf-8') as f:
    event_type_list = [item.strip() for item in f.readlines()]

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, event_type_list, eve_extend_map, 'alpaca_33B_en')

merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, event_type_list, 'alpaca_33B_en')

Llama2-CASIE

In [11]:
with open('./results/details_llama2_70B_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_llama2_70B_en.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/casie/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/casie/final_event_extend_map_llama2_70B_en.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('../CASIE/processed_data/labels.txt','r',encoding='utf-8') as f:
    event_type_list = [item.strip() for item in f.readlines()]

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, event_type_list, eve_extend_map, 'llama2_70B_en')

merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, event_type_list,'llama2_70B_en')

ChatGLM-DuEE

In [6]:
with open('./results/details_chatglm_6B_zh.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_chatglm_6B_zh.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/duee/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/duee/final_event_extend_map_chatglm_6B_zh.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('../DuEE/processed_data/labels.txt','r',encoding='utf-8') as f:
    event_type_list = [item.strip() for item in f.readlines()]

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, event_type_list, eve_extend_map, 'chatglm_6B_zh')

merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, event_type_list, 'chatglm_6B_zh')

ChatGLM-CASIE

In [15]:
with open('./results/details_chatglm_6B_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_chatglm_6B_en.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/casie/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/casie/final_event_extend_map_chatglm_6B_en.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('../CASIE/processed_data/labels.txt','r',encoding='utf-8') as f:
    event_type_list = [item.strip() for item in f.readlines()]

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, event_type_list, eve_extend_map, 'chatglm_6B_en')

merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, event_type_list, 'chatglm_6B_en')

GPT4-DuEE

In [ ]:
with open('./results/details_gpt4_zh.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./data/duee/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/duee/final_event_extend_map_gpt4_zh.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('../DuEE/processed_data/labels.txt','r',encoding='utf-8') as f:
    event_type_list = [item.strip() for item in f.readlines()]

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, event_type_list, eve_extend_map, 'gpt4_zh')

GPT4-CASIE

In [ ]:
with open('./results/details_gpt4_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./data/casie/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/casie/final_event_extend_map_gpt4_en.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('../CASIE/processed_data/labels.txt','r',encoding='utf-8') as f:
    event_type_list = [item.strip() for item in f.readlines()]

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, event_type_list, eve_extend_map, 'gpt4_en')

Baichuan2-DuEE

In [7]:
with open('./results/details_baichuan2_13B_zh.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_baichuan2_13B_zh.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/duee/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/duee/final_event_extend_map_baichuan2_13B_zh.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('../DuEE/processed_data/labels.txt','r',encoding='utf-8') as f:
    event_type_list = [item.strip() for item in f.readlines()]

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, event_type_list, eve_extend_map, 'baichuan2_13B_zh')

merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, event_type_list, 'baichuan2_13B_zh')

Baichuan2-CASIE

In [8]:
with open('./results/details_baichuan2_13B_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_baichuan2_13B_en.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/casie/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/casie/final_event_extend_map_baichuan2_13B_en.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

with open('../CASIE/processed_data/labels.txt','r',encoding='utf-8') as f:
    event_type_list = [item.strip() for item in f.readlines()]

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, event_type_list, eve_extend_map, 'baichuan2_13B_en')

merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, event_type_list, 'baichuan2_13B_en')